In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from panoradio_hf.data import IQDataModel
from panoradio_hf.allconvnet import AllConvNet
from panoradio_hf.net_utils import initialize_pl_trainer
import lightning as pl

In [ ]:
iqdata = IQDataModel()
model = AllConvNet()

callbacks, mlf_logger = initialize_pl_trainer("all-conv-net")

trainer = pl.Trainer(callbacks=callbacks,
                     max_epochs=100,
                     logger=mlf_logger)

trainer.fit(model, iqdata)